In [1]:
import os
import sys
from datasets import load_dataset 
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig 
from transformers import pipeline
print('ready loading data')

# Check Python version
print("Python version:", sys.version)

# Check Conda environment
conda_env = os.environ.get('CONDA_DEFAULT_ENV')
print("Conda environment:", conda_env)

ready loading data
Python version: 3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]
Conda environment: base


In [2]:
from transformers import pipeline 

# generation pipeline
generator = pipeline('text-generation',model='gpt2')

# example of text generation
result=generator("Once upon a time,")
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Once upon a time, when we were under great pressure. We got to a place that was more than ten thousand square metres. In the course of a short period of time, it became the most-loved city in our land. We saw'}]


In [3]:
llm =  pipeline('text-generation')
prompt = "New York city is famous for"
outputs = llm(prompt, max_length=150)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\bciez\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


New York city is famous for its urban centers and can be hard to find in the surrounding areas. Luckily with the introduction of the New York's Metropolitan Transit Authority, its location and popularity are increasing exponentially. There are now more "Bikers" and "Migrant" sections in every city.

These are actually very much in a city of 9 million unique people. The main difference between the cities in California and Florida or Colorado and their regional neighbors is that Florida and Florida are the most progressive areas in their respective states. They all have their own city hall, parks, public transportation and they have their own budgeting system. It doesn't really matter in terms of what the city gets their money for, they can make up their own budget
